In [217]:
# part 1
import pandas as pd
from collections import Counter 

f = open("input.txt", "r")
contents = f.read()
f.close()

lines = [line for line in contents.split("\n") if line]

def get_adj_datetime(line):
    date_str = line[1:17]
    year_str = date_str[:4]
    adj_year = str(int(year_str)+500)
    adj_date_str = date_str.replace(year_str, adj_year)
    return pd.to_datetime(adj_date_str)

lines_w_datetime = [{"datetime": get_adj_datetime(line), "line": line} for line in lines]
lines_w_datetime.sort(key= lambda x: x["datetime"])
sorted_lines = [x["line"] for x in lines_w_datetime]

def get_guard_id(line):
    return line.split(" ")[3]

data = []
obj = {}
for x in lines_w_datetime:
    line, minute = x["line"], x["datetime"].minute
    if "Guard" in line:
        if obj:
            data.append(obj)
        obj = {"id": get_guard_id(line), "state_change": []}
        continue
    if "asleep" in line or "wakes" in line:
        obj["state_change"].append(minute)
        continue
data.append(obj)

for obj in data:
    awake_state = True
    obj["awake"] = []
    for minute in range(60):
        if minute not in obj["state_change"]:
            obj["awake"].append(awake_state)
            continue
        awake_state = not awake_state
        obj["awake"].append(awake_state)
        
sleep_duration = {}
for obj in data:
    if obj["id"] not in sleep_duration:
        sleep_duration[obj["id"]] = 0
    sleep_duration[obj["id"]] += len([state for state in obj["awake"] if not state])
    
duration_list = list(sleep_duration.items())
duration_list.sort(key= lambda x: x[1])
sleepiest_guard = duration_list[-1][0]

def get_sleeping_minutes(obj):
    return [minute for minute, awake in enumerate(obj["awake"]) if not awake]

sleepiest_guard_asleep = []
for obj in data:
    if obj["id"] == sleepiest_guard:
        sleepiest_guard_asleep += get_sleeping_minutes(obj)
        
sleep_counts = list(Counter(sleepiest_guard_asleep).items())
sleep_counts.sort(key=lambda x: x[1])
sleepiest_minute = sleep_counts[-1][0]

print(sleepiest_minute*int(sleepiest_guard[1:]))

# part 2

sleeping_minutes_dict = {}
for obj in data:
    if obj["id"] not in sleeping_minutes_dict:
        sleeping_minutes_dict[obj["id"]] = []
    sleeping_minutes_dict[obj["id"]] += get_sleeping_minutes(obj)
    
leaderboard = []
for guard_id in sleeping_minutes_dict:
    asleep = sleeping_minutes_dict[guard_id]
    if not asleep:
        continue
    sleep_counts = list(Counter(asleep).items())
    sleep_counts.sort(key=lambda x: x[1], reverse=True)    
    leaderboard.append({"id": guard_id, "count": sleep_counts[0][1], "sleepiest_minute":  sleep_counts[0][0]})
    
leaderboard.sort(key=lambda x: x["count"], reverse=True)
sleep_counts.sort(key=lambda x: x[1], reverse=True)
print(int(leaderboard[0]["id"][1:])*leaderboard[0]["sleepiest_minute"])

84834


53427
